In [1]:
import torch
import cv2
from ultralytics import YOLO
from bytetrack import BYTETracker  # Import ByteTrack - ensure you have the bytetrack library

# Load YOLOv8 model pretrained for detection
model = YOLO("yolov8n.pt")  # 'yolov8n.pt' is a small, fast model, you can replace with 'yolov8x.pt' for better accuracy

# Initialize the ByteTrack tracker
tracker = BYTETracker()

# Define the video source: 0 for webcam, or replace with a URL or file path for other video sources
video_source = 0  # Change to URL or file path if needed
cap = cv2.VideoCapture(video_source)

# Check if the video source is opened successfully
if not cap.isOpened():
    print("Error: Could not open video source.")
    exit()

# Process the video frame by frame
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Get the dimensions of the frame
    height, width, _ = frame.shape

    # Use YOLO model to perform object detection
    results = model(frame)

    # Extract bounding boxes, scores, and class IDs from detection results
    bboxes = results.xyxy[0][:, :4].cpu().numpy()  # Bounding boxes
    scores = results.xyxy[0][:, 4].cpu().numpy()  # Confidence scores
    class_ids = results.xyxy[0][:, 5].cpu().numpy().astype(int)  # Class IDs

    # Filter detections by score threshold to reduce false positives
    min_score_threshold = 0.4
    valid_idx = scores > min_score_threshold
    bboxes = bboxes[valid_idx]
    scores = scores[valid_idx]
    class_ids = class_ids[valid_idx]

    # Update tracker with the detected bounding boxes and their scores
    tracks = tracker.update(bboxes, scores, class_ids, width, height)

    # Draw tracked objects on the frame
    for track in tracks:
        track_id = track.track_id
        x1, y1, x2, y2 = track.tlbr  # Bounding box coordinates
        class_name = model.names[track.class_id]

        # Draw the bounding box and track ID
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(
            frame,
            f"{class_name} ID: {track_id}",
            (int(x1), int(y1) - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (0, 255, 0),
            2,
        )

    # Display the frame
    cv2.imshow("Object Tracking", frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release the video capture object and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'bytetrack'

In [2]:
!pip install bytetrack

  Using cached bytetrack-0.0.1.tar.gz (100 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [123 lines of output]
      running bdist_wheel
      /home/amehmood/anaconda3/envs/main_env/lib/python3.8/site-packages/torch/utils/cpp_extension.py:499: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
        warnings.warn(msg.format('we could not find ninja.'))
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-38
      creating build/lib.linux-x86_64-cpython-38/bytetrack
      copying bytetrack/__init__.py -> build/lib.linux-x86_64-cpython-38/bytetrack
      creating build/lib.linux-x86_64-cpython-38/bytetrack/motdt_tracker
      copying bytetrack/motdt_tracker/reid_model.py -> build/lib.linux-x86_64-cpython-38/bytetrack/

In [3]:
import torch
import cv2
from ultralytics import YOLO
from bytetrack import BYTETracker  # Import ByteTrack - ensure you have the bytetrack library installed

# Load YOLOv8 model pretrained for detection
model = YOLO("yolov8n.pt")  # You can replace 'yolov8n.pt' with other versions like 'yolov8x.pt' for better accuracy

# Initialize the ByteTrack tracker
tracker = BYTETracker()

# Define the video source: 0 for webcam, or replace with a URL or file path for other video sources
video_source = 0  # Change to URL or file path if needed
cap = cv2.VideoCapture(video_source)

# Check if the video source is opened successfully
if not cap.isOpened():
    print("Error: Could not open video source.")
    exit()

# Get frame dimensions and frame rate of the input video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create a VideoWriter object to save the output video
output_filename = "output_tracked_video.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Codec for .mp4 files
out = cv2.VideoWriter(output_filename, fourcc, fps, (frame_width, frame_height))

# Process the video frame by frame
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Get the dimensions of the frame
    height, width, _ = frame.shape

    # Use YOLO model to perform object detection
    results = model(frame)

    # Extract bounding boxes, scores, and class IDs from detection results
    bboxes = results.xyxy[0][:, :4].cpu().numpy()  # Bounding boxes
    scores = results.xyxy[0][:, 4].cpu().numpy()  # Confidence scores
    class_ids = results.xyxy[0][:, 5].cpu().numpy().astype(int)  # Class IDs

    # Filter detections by score threshold to reduce false positives
    min_score_threshold = 0.4
    valid_idx = scores > min_score_threshold
    bboxes = bboxes[valid_idx]
    scores = scores[valid_idx]
    class_ids = class_ids[valid_idx]

    # Update tracker with the detected bounding boxes and their scores
    tracks = tracker.update(bboxes, scores, class_ids, width, height)

    # Draw tracked objects on the frame
    for track in tracks:
        track_id = track.track_id
        x1, y1, x2, y2 = track.tlbr  # Bounding box coordinates
        class_name = model.names[track.class_id]

        # Draw the bounding box and track ID
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(
            frame,
            f"{class_name} ID: {track_id}",
            (int(x1), int(y1) - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (0, 255, 0),
            2,
        )

    # Write the frame with tracked objects to the output video
    out.write(frame)

    # Display the frame (optional, can be commented out if running on a server without display)
    cv2.imshow("Object Tracking", frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release the video capture, video writer objects, and close all OpenCV windows
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Output video saved as {output_filename}")


ModuleNotFoundError: No module named 'bytetrack'